<a href="https://colab.research.google.com/github/A-shazli/Brain_Tumor_seg_GP/blob/main/nnunet_brain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Import Packages for Google Colab

In [ ]:
# Import basic packages for later use
import os
import shutil
from collections import OrderedDict

import json
import matplotlib.pyplot as plt
import nibabel as nib

import numpy as np
import torch

In [ ]:
# check whether GPU accelerated computing is available
assert torch.cuda.is_available() # if there is an error here, enable GPU in the Runtime

#  Installing nnU-Net

In [ ]:
# install nnunet 
!pip install nnunet 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 251 kB 14.5 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 151 kB 51.7 MB/s 
     |████████████████████████████████| 61 kB 391 kB/s 
     |████████████████████████████████| 52.8 MB 197 kB/s 
     |████████████████████████████████| 96 kB 6.2 MB/s 
     |████████████████████████████████| 2.0 MB 49.3 MB/s 
     |████████████████████████████████| 13.0 MB 49.3 MB/s 
  Created wheel for nnunet: filename=nnunet-1.7.0-py3-none-any.whl size=487821 sha256=09fc2b7f1eca69df16f6134b2aa3ea5d8c0d26373b570842c2d93e85616e19b3
  Stored in directory: /root/.cache/pip/wheels/66/f6/19/c51389976a9ae30212178d1c41497893504b900f043fe1c665
  Created wheel for batchgenerators: filename=batchgenerators-0.24-py3-none-any.whl size=89209 sha256=de0ed4918d66f5c01a089ec5f6279f9ac7189a62fff2faa1598a8918fa118f7f
  Stored in direc

**Note**: you do not have to restart the Runtime even when the following error appears, this is simply due to a reinstallation of a package - so no worries. 


```
WARNING: The following packages were previously imported in this runtime:

[argparse]

You must restart the runtime in order to use newly installed versions.
```

# friendly warning !!!!!!!
the nnunet in this example will run for 1000 epochs this is the default of nnunet but you can change that by going to the folders icon in colab from there 

usr---->local----->lib----->python3.7------>dist-packages
----->nnunet---->training---->network_training----->nnUNetTrainerV2.py

double click this file it will open in colab in the __init__ function there is a variable __max_number_of_epochs__ change it's value from 1000 to 2  then ctrl+s and finally restart the runtime and import the packages and DON'T INSTALL THE NNUNET AGAIN  just skip the instllation cell and continue the notebook

# Verifying installation of nn-Unet

In [ ]:
# check if nnunet can be imported
import nnunet




Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



You should read the following if the installation was successful:


```
Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

```



# 4. Connect Google Colab with GoogleDrive
This is heavily encouraged when using Google Colab, otherwise loading, accessing and saving of data checkpoints etc. is just much harder (if not impossible).

**Note**:
When ```drive.mount()``` is executed, there will appear a popup window, which will ask you which Google Account should be connected and then for permission regarding Colab accessing your GoogleDrive.

There can be some problems at this step, for me the solution was to Download the GoogleDrive Application.
You can download it from: https://www.google.com/drive/download/


If there are any problems at this stage, feel free to contact me 

In [ ]:
# for mounting the drive

from google.colab import drive
drive.mount('/content/drive',force_remount = True)

drive_dir = "/content/drive/My Drive"
mount_dir = os.path.join(drive_dir, "Colab Notebooks")
base_dir = os.getcwd()

Mounted at /content/drive


In [ ]:
assert os.path.exists(drive_dir) # if this fails, something went wrong with mounting GoogleDrive
if os.path.exists(mount_dir) is False:
    os.makedirs(mount_dir)

#  Setting up nnU-Nets folder structure and environment variables
nnUnet expects a certain folder structure and environment variables. 

Roughly they tell nnUnet:
1. Where to look for stuff
2. Where to put stuff

For more information about this please check: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/setting_up_paths.md

In [ ]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input: 
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder 
    """
    if os.path.exists(folder_path):
        
        if not overwrite:
            print(f"{folder_path} exists.")
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
      os.makedirs(folder_path)
      print(f"{folder_path} created!")

##  Set environment Variables and creating folders

In [ ]:
# Maybe move path of preprocessed data directly on content - this may be signifcantely faster!
print("Current Working Directory {}".format(os.getcwd()))
path_dict = {
    "nnUNet_raw_data_base" : os.path.join(mount_dir, "nnUNet_raw_data_base"), 
    # "nnUNet_preprocessed" : os.path.join(mount_dir, "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 112s
    "nnUNet_preprocessed" : os.path.join(base_dir, "nnUNet_preprocessed"), # 1 experiment: 1 epoch took 108s -> seems faster take this
    "RESULTS_FOLDER" : os.path.join(mount_dir, "nnUNet_Results_Folder"),
        "RAW_DATA_PATH" : os.path.join(mount_dir, "RawData"), 
}

# Write paths to environment variables
for env_var, path in path_dict.items():
  os.environ[env_var] = path 

# Check whether all environment variables are set correct!
for env_var, path in path_dict.items():
  if os.getenv(env_var) != path:
    print("Error:")
    print("Environment Variable {} is not set correctly!".format(env_var))
    print("Should be {}".format(path))
    print("Variable is {}".format(os.getenv(env_var)))
  make_if_dont_exist(path, overwrite=False)

print("If No Error Occured Continue Forward. =)")

Current Working Directory /content
/content/drive/My Drive/Colab Notebooks/nnUNet_raw_data_base exists.
/content/nnUNet_preprocessed exists.
/content/drive/My Drive/Colab Notebooks/nnUNet_Results_Folder exists.
/content/drive/My Drive/Colab Notebooks/RawData exists.
If No Error Occured Continue Forward. =)


In [ ]:
os.chdir(path_dict["RAW_DATA_PATH"])
!unzip "Path to zipped data"
os.chdir(base_dir)

Archive:  /content/drive/MyDrive/Colab Notebooks/RawData/Sample BraTS21 50 Examples.zip
replace Sample BraTS21 50 Examples/BraTS2021_00000/BraTS2021_00000_flair.nii.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Sample BraTS21 50 Examples/BraTS2021_00000/BraTS2021_00000_flair.nii.gz  
  inflating: Sample BraTS21 50 Examples/BraTS2021_00000/BraTS2021_00000_seg.nii.gz  
  inflating: Sample BraTS21 50 Examples/BraTS2021_00000/BraTS2021_00000_t1.nii.gz  
  inflating: Sample BraTS21 50 Examples/BraTS2021_00000/BraTS2021_00000_t1ce.nii.gz  
  inflating: Sample BraTS21 50 Examples/BraTS2021_00000/BraTS2021_00000_t2.nii.gz  
  inflating: Sample BraTS21 50 Examples/BraTS2021_00002/BraTS2021_00002_flair.nii.gz  
  inflating: Sample BraTS21 50 Examples/BraTS2021_00002/BraTS2021_00002_seg.nii.gz  
  inflating: Sample BraTS21 50 Examples/BraTS2021_00002/BraTS2021_00002_t1.nii.gz  
  inflating: Sample BraTS21 50 Examples/BraTS2021_00002/BraTS2021_00002_t1ce.nii.gz  
  inflating: Sample Bra

In [ ]:
from batchgenerators.utilities.file_and_folder_operations import *
from nnunet.paths import nnUNet_raw_data
import SimpleITK as sitk
import shutil


def copy_BraTS_segmentation_and_convert_labels(in_file, out_file):
    # use this for segmentation only!!!
    # nnUNet wants the labels to be continuous. BraTS is 0, 1, 2, 4 -> we make that into 0, 1, 2, 3
    img = sitk.ReadImage(in_file)
    img_npy = sitk.GetArrayFromImage(img)

    uniques = np.unique(img_npy)
    for u in uniques:
        if u not in [0, 1, 2, 4]:
            raise RuntimeError('unexpected label')

    seg_new = np.zeros_like(img_npy)
    seg_new[img_npy == 4] = 3
    seg_new[img_npy == 2] = 1
    seg_new[img_npy == 1] = 2
    img_corr = sitk.GetImageFromArray(seg_new)
    img_corr.CopyInformation(img)
    sitk.WriteImage(img_corr, out_file)


if __name__ == "__main__":
    """
    REMEMBER TO CONVERT LABELS BACK TO BRATS CONVENTION AFTER PREDICTION!
    """

    task_name = "Task501_BraTS2021"
    downloaded_data_dir = "path to unzipped data"

    target_base = join(nnUNet_raw_data, task_name)
    target_imagesTr = join(target_base, "imagesTr")
    target_imagesVal = join(target_base, "imagesVal")
    target_imagesTs = join(target_base, "imagesTs")
    target_labelsTr = join(target_base, "labelsTr")

    maybe_mkdir_p(target_imagesTr)
    maybe_mkdir_p(target_imagesVal)
    maybe_mkdir_p(target_imagesTs)
    maybe_mkdir_p(target_labelsTr)

    patient_names = []

    for p in subdirs(downloaded_data_dir, join=False):
        patdir = join(downloaded_data_dir, p)
        patient_name = p
        patient_names.append(patient_name)
        t1 = join(patdir, p + "_t1.nii.gz")
        t1c = join(patdir, p + "_t1ce.nii.gz")
        t2 = join(patdir, p + "_t2.nii.gz")
        flair = join(patdir, p + "_flair.nii.gz")
        seg = join(patdir, p + "_seg.nii.gz")

        assert all([
            isfile(t1),
            isfile(t1c),
            isfile(t2),
            isfile(flair),
            isfile(seg)
        ]), "%s" % patient_name

        shutil.copy(t1, join(target_imagesTr, patient_name + "_0000.nii.gz"))
        shutil.copy(t1c, join(target_imagesTr, patient_name + "_0001.nii.gz"))
        shutil.copy(t2, join(target_imagesTr, patient_name + "_0002.nii.gz"))
        shutil.copy(flair, join(target_imagesTr, patient_name + "_0003.nii.gz"))

        copy_BraTS_segmentation_and_convert_labels(seg, join(target_labelsTr, patient_name + ".nii.gz"))


    json_dict = OrderedDict()
    json_dict['name'] = "BraTS2021"
    json_dict['description'] = "nothing"
    json_dict['tensorImageSize'] = "4D"
    json_dict['reference'] = "see BraTS2021"
    json_dict['licence'] = "see BraTS2021 license"
    json_dict['release'] = "0.0"
    json_dict['modality'] = {
        "0": "T1",
        "1": "T1ce",
        "2": "T2",
        "3": "FLAIR"
    }
    json_dict['labels'] = {
        "0": "background",
        "1": "edema",
        "2": "non-enhancing",
        "3": "enhancing",
    }
    json_dict['numTraining'] = len(patient_names)
    json_dict['numTest'] = 0
    json_dict['training'] = [{'image': "./imagesTr/%s.nii.gz" % i, "label": "./labelsTr/%s.nii.gz" % i} for i in
                             patient_names]
    json_dict['test'] = []

    save_json(json_dict, join(target_base, "dataset.json"))

    # downloaded_data_dir = "/home/sdp/MLPERF/Brats2019_DATA/MICCAI_BraTS_2019_Data_Validation"
    #
    # for p in subdirs(downloaded_data_dir, join=False):
    #     patdir = join(downloaded_data_dir, p)
    #     patient_name = p
    #     t1 = join(patdir, p + "_t1.nii.gz")
    #     t1c = join(patdir, p + "_t1ce.nii.gz")
    #     t2 = join(patdir, p + "_t2.nii.gz")
    #     flair = join(patdir, p + "_flair.nii.gz")
    #
    #     assert all([
    #         isfile(t1),
    #         isfile(t1c),
    #         isfile(t2),
    #         isfile(flair),
    #     ]), "%s" % patient_name
    #
    #     shutil.copy(t1, join(target_imagesVal, patient_name + "_0000.nii.gz"))
    #     shutil.copy(t1c, join(target_imagesVal, patient_name + "_0001.nii.gz"))
    #     shutil.copy(t2, join(target_imagesVal, patient_name + "_0002.nii.gz"))
    #     shutil.copy(flair, join(target_imagesVal, patient_name + "_0003.nii.gz"))

    """
    #I dont have the testing data
    downloaded_data_dir = "/home/fabian/Downloads/BraTS2018_train_val_test_data/MICCAI_BraTS_2018_Data_Testing_FIsensee"

    for p in subdirs(downloaded_data_dir, join=False):
        patdir = join(downloaded_data_dir, p)
        patient_name = p
        t1 = join(patdir, p + "_t1.nii.gz")
        t1c = join(patdir, p + "_t1ce.nii.gz")
        t2 = join(patdir, p + "_t2.nii.gz")
        flair = join(patdir, p + "_flair.nii.gz")

        assert all([
            isfile(t1),
            isfile(t1c),
            isfile(t2),
            isfile(flair),
        ]), "%s" % patient_name

        shutil.copy(t1, join(target_imagesTs, patient_name + "_0000.nii.gz"))
        shutil.copy(t1c, join(target_imagesTs, patient_name + "_0001.nii.gz"))
        shutil.copy(t2, join(target_imagesTs, patient_name + "_0002.nii.gz"))
        shutil.copy(flair, join(target_imagesTs, patient_name + "_0003.nii.gz"))"""


## Extracting Rule Based Parameters
This will preprocess the dataset to allow fast training and saves it into the "nnUNet_preprocessed" folder.
Further rule based parameters will be extracted in the planning step.

In [ ]:
!nnUNet_plan_and_preprocess -t 501 --verify_dataset_integrity



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Verifying training set
checking case BraTS2021_00000
checking case BraTS2021_00002
checking case BraTS2021_00003
checking case BraTS2021_00005
checking case BraTS2021_00006
checking case BraTS2021_00008
checking case BraTS2021_00009
checking case BraTS2021_00011
checking case BraTS2021_00012
checking case BraTS2021_00014
checking case BraTS2021_00016
checking case BraTS2021_00017
checking case BraTS2021_00018
checking case BraTS2021_00019
checking case BraTS2021_00020
checking case BraTS2021_00021
checking case BraTS2021_00022
checking case BraTS2021_00024
checking case BraTS2021_00025
checking case BraTS2021_00026
checking case B

In [ ]:
# train the 2d nnUnet with Task 501 and Cross Validation Split 0
!nnUNet_train 2d nnUNetTrainerV2 501 0

# training the Full Model will take quite a while - therefore it is advised to interrupt the training after some time



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

###############################################
I am running the following nnUNet: 2d
My trainer class is:  <class 'nnunet.training.network_training.nnUNetTrainerV2.nnUNetTrainerV2'>
For that I will be using the following configuration:
num_classes:  3
modalities:  {0: 'T1', 1: 'T1ce', 2: 'T2', 3: 'FLAIR'}
use_mask_for_norm OrderedDict([(0, True), (1, True), (2, True), (3, True)])
keep_only_largest_region None
min_region_size_per_class None
min_size_per_class None
normalization_schemes OrderedDict([(0, 'nonCT'), (1, 'nonCT'), (2, 'nonCT'), (3, 'nonCT')])
stages...

stage:  0
{'batch_size': 106, 'num_pool_per_axis': [5, 5], 'patch_

## prediction using our model 

In [ ]:
!nnUNet_predict   -i "/content/drive/MyDrive/Colab Notebooks/nnUNet_raw_data_base/nnUNet_raw_data/Task043_BraTS2021/imagesTr"  -o "/content/drive/MyDrive/Colab Notebooks/nnUNet_raw_data_base/nnUNet_raw_data/Task043_BraTS2021/imagesTs" -t 43  -m 2d



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /content/drive/My Drive/Colab Notebooks/nnUNet_Results_Folder/nnUNet/2d/Task043_BraTS2021/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 4 input modalities for each image
Found 10 unique case ids, here are some examples: ['BraTS2021_00014' 'BraTS2021_00012' 'BraTS2021_00014' 'BraTS2021_00012'
 'BraTS2021_00011' 'BraTS2021_00008' 'BraTS2021_00012' 'BraTS2021_00009'
 'BraTS2021_00009' 'BraTS2021_00006']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 10
number of cases that still need to be predicted: 10
emptying cuda cache
loading p